In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import numpy as np
from load_gan import load_gan
from load_cases import load_cases, load_save_name
from train_gan import train_gan
from evaluation import evaluation
from keras.datasets import mnist

os.environ["KERAS_BACKEND"] = "tensorflow"

Using TensorFlow backend.


In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5)/127.5
x_test = (x_test.astype(np.float32) - 127.5)/127.5
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [ ]:
epochs = 4
randomDim = 50
batchSize = 128
examples = 100
cases = load_cases(loss_bases=["Goodfellow"],
                   link_modes=["alpha"],
                   alphas=[1.],
                   offsets=[0.])

In [ ]:
for case in cases:
    print(case)
    save_name = load_save_name(case)
    generator, discriminator, gan = load_gan(offset=case["offset"],
                                             alpha=case["alpha"],
                                             randomDim=randomDim,
                                             link_mode=case["link_mode"],
                                             power=case["power"],
                                             mult=case["mult"],
                                             sqrt=case["sqrt"],
                                             loss_base=case["loss_base"])

    to_be_trusted, disc_loss, gen_loss, hurting = train_gan(disc=discriminator,
                                                            gen=generator,
                                                            gan=gan,
                                                            x_train=x_train,
                                                            epochs=epochs,
                                                            batch_size=batchSize,
                                                            d_loss_limit=0.05,
                                                            randomDim=randomDim)
    if to_be_trusted:
        location = "results/" + case["loss_base"] + "/" + case["link_mode"] + "/" + save_name
        evaluation(generator=generator,
                   randomDim=randomDim,
                   location=location,
                   examples=examples, title=save_name,
                   hurting=hurting, gen_loss=gen_loss, disc_loss=disc_loss, x_test=x_test[:examples*examples])

print("LETS GO TO SLEEP \n "*10)

{'link_mode': 'alpha', 'loss_base': 'Goodfellow', 'alpha': 1.0, 'offset': 0.0, 'power': None, 'mult': None, 'sqrt': None}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/peseux/Desktop/gitELECOM/NovGan/load_gan.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=50, kernel_initializer=<keras.ini..., kernel_constraint=None)`
  W_constraint=W_constraint))
/home/peseux/Desktop/gitELECOM/NovGan/load_gan.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_constraint=None)`
  W_constraint=W_constraint))
/home/peseux/Desktop/gitELECOM/NovGan/load_gan.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, kernel_constraint=None)`
  W_constraint=W_constraint))
/home/peseux/Desktop/gitELECOM/NovGan/load_gan.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(784, activation="tanh", kernel_constraint=None)`
  activation=activation))
/home/peseux/Desktop/gitELECOM/NovGan/load_gan.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, input_dim=784, kernel_initializer=<keras.ini..., kernel_constraint=None)`
  W_constraint=W_cons

Instructions for updating:
Use tf.cast instead.


 50%|█████     | 2/4 [01:20<01:21, 40.67s/it]

In [ ]:
noise = np.random.normal(0, 1, size=[100, randomDim])

In [ ]:
images = generator.predict(noise)

In [ ]:
discriminator.predict(images)

In [ ]:
discriminator.evaluate(images, np.ones(100))

In [ ]:
gan.evaluate(noise, np.zeros(100))

In [ ]:
images

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def plotImages(generatedImages, dim=(10,10), title="title"):
    plt.figure(figsize=dim)
    plt.title(title)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
plotImages(images.reshape(100, 28,28))

In [ ]:
batch_size = 10

In [ ]:
yDis = np.zeros(2 * batch_size)
yDis[:batch_size] = 1

In [ ]:
yDis

In [ ]:
disc_loss

In [ ]:
plt.plot(gen_loss, label="gen")
plt.plot(disc_loss, label="disc")
plt.legend()
plt.show()

In [ ]:
gen_loss